In [1]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

/usr/local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/usr/local/lib/python3.11/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [4]:
with open("training/cc_cols.txt", "r") as f:
    cc_cols = f.readlines()
cc_cols = list(map(lambda x:x[:-1], cc_cols))

with open("training/pmh_cols.txt", "r") as f:
    pmh_cols = f.readlines()
pmh_cols = list(map(lambda x:x[:-1], pmh_cols))

In [5]:
import numpy as np

In [6]:
cc_embeddings = np.load("OpenAIEmbeddings/cc_embeddings.npy")
pmh_embeddings = np.load("OpenAIEmbeddings/pmh_embeddings.npy")

In [7]:
sev_words = ['heart attack', 'seizure', 'stroke']

sev_encodings = model.encode(sev_words)

In [8]:
cc_sev_scores = np.zeros((cc_embeddings.shape[0], len(sev_words)))
pmh_sev_scores = np.zeros((pmh_embeddings.shape[0], len(sev_words)))

for i in range(len(sev_words)):
    for j in range(cc_embeddings.shape[0]):
        cc_sev_scores[j][i] = np.dot(cc_embeddings[j], sev_encodings[i]) / (np.linalg.norm(cc_embeddings[j]) * np.linalg.norm(sev_encodings[i]))

    for j in range(pmh_embeddings.shape[0]):
        pmh_sev_scores[j][i] = np.dot(pmh_embeddings[j], sev_encodings[i]) / (np.linalg.norm(pmh_embeddings[j]) * np.linalg.norm(sev_encodings[i]))

In [9]:
cc_sev_scores = list(np.mean(cc_sev_scores, axis=1))
pmh_sev_scores = list(np.mean(pmh_sev_scores, axis=1))

In [10]:
cc_sev_scores = [(i, cc_sev_scores[i]) for i in range(len(cc_sev_scores))]
pmh_sev_scores = [(i, pmh_sev_scores[i]) for i in range(len(pmh_sev_scores))]

cc_sev_scores = sorted(cc_sev_scores, key=lambda x:x[1], reverse=True)
pmh_sev_scores = sorted(pmh_sev_scores, key=lambda x:x[1], reverse=True)

In [11]:
def get_sev_index(word):
    idx_orders = [i[0] for i in cc_sev_scores]
    return idx_orders.index(cc_cols.index(word))

In [17]:
pmh_sev_scores

[(219, 0.47696658968925476),
 (257, 0.43785298864046734),
 (16, 0.4250001907348633),
 (40, 0.4020434121290843),
 (229, 0.39613216121991474),
 (65, 0.3897955318291982),
 (220, 0.3894064923127492),
 (28, 0.34811731179555255),
 (87, 0.33681729932626087),
 (63, 0.3336652119954427),
 (249, 0.32134702801704407),
 (37, 0.31941769520441693),
 (115, 0.3164738516012828),
 (39, 0.3155514846245448),
 (117, 0.315414622426033),
 (38, 0.31427015364170074),
 (233, 0.3133492171764374),
 (89, 0.3114054203033447),
 (159, 0.3103126287460327),
 (32, 0.3097518980503082),
 (234, 0.3082518329222997),
 (94, 0.3074888239304225),
 (260, 0.30645625789960224),
 (107, 0.30598461627960205),
 (262, 0.30458510915438336),
 (17, 0.30232160290082294),
 (31, 0.3021032164494197),
 (86, 0.29611314336458844),
 (240, 0.29517563184102374),
 (109, 0.29496446748574573),
 (228, 0.28892282644907635),
 (19, 0.287579283118248),
 (21, 0.2868981758753459),
 (8, 0.2841538389523824),
 (164, 0.2790655891100566),
 (271, 0.2788049081961314